### Imports, constants and settings

In [22]:
from typing import Tuple, Optional, Any
import hashlib
import binascii

In [23]:

# Set DEBUG to True to get a detailed debug output including
# intermediate values during key generation, signing, and
# verification. This is implemented via calls to the
# debug_print_vars() function.
#
# If you want to print values on an individual basis, use
# the pretty() function, e.g., print(pretty(foo)).
DEBUG = True

In [24]:

p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Points are tuples of X and Y coordinates and the point at infinity is
# represented by the None keyword.
G = (0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798, 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8)


In [25]:
p

115792089237316195423570985008687907853269984665640564039457584007908834671663

In [26]:
n

115792089237316195423570985008687907852837564279074904382605163141518161494337

In [27]:
G

(55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)

In [28]:
Point = Tuple[int, int]
Point

typing.Tuple[int, int]

In [29]:
# This implementation can be sped up by storing the midstate after hashing
# tag_hash instead of rehashing it all the time.
def tagged_hash(tag: str, msg: bytes) -> bytes:
    tag_hash = hashlib.sha256(tag.encode()).digest()
    return hashlib.sha256(tag_hash + tag_hash + msg).digest()


In [30]:
tag = "example_tag"
message = b"This is an example message"

tagged_hash_value = tagged_hash(tag, message)
tagged_hash_value.hex()

'379e0663cf310fb6b228f429f190aa1fe3c0b4ac37fe603a8b1a30fa025e9d34'

In [31]:
def is_infinite(P: Optional[Point]) -> bool:
    return P is None

In [32]:
P1: Optional[Point] = (1, 2)
P2: Optional[Point] = None

print(is_infinite(P1)) # Output: False
print(is_infinite(P2)) # Output: True

False
True


In [33]:
def x(P: Point) -> int:
    assert not is_infinite(P)
    return P[0]

# if is_infinite, returns assertionerror
# else returns P[0]

In [34]:
def y(P: Point) -> int:
    assert not is_infinite(P)
    return P[1]

# if is_infinite, returns assertionerror
# else returns P[1]

In [35]:
# point addition function
def point_add(P1: Optional[Point], P2: Optional[Point]) -> Optional[Point]:
    if P1 is None:
        return P2
    # returns p2 if p1 is none
    if P2 is None:
        return P1
    # returns p1 if p2 is none
    if (x(P1) == x(P2)) and (y(P1) != y(P2)):
        return None
    # if x(p1) and x(p2) are equal, and y(p1) and y(p2) arent equal, return none 
    # This corresponds to a situation where the two points are vertically aligned and cannot be added together.
    if P1 == P2:
        lam = (3 * x(P1) * x(P1) * pow(2 * y(P1), p - 2, p)) % p
    else:
        lam = ((y(P2) - y(P1)) * pow(x(P2) - x(P1), p - 2, p)) % p
    x3 = (lam * lam - x(P1) - x(P2)) % p
    return (x3, (lam * (x(P1) - x3) - y(P1)) % p)

In [36]:
# multiply a point by an integer, uses the point_add function
def point_mul(P: Optional[Point], n: int) -> Optional[Point]:
    R = None
    for i in range(256):
        if (n >> i) & 1:
            R = point_add(R, P)
        P = point_add(P, P)
    return R